What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [ ]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime

In [ ]:
# Read in postcode dataset
SWW_postcode = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\SWW Nitrate 2000-PRESENT_postcode.csv")

- Get rid of the white space in the postcodes that have XXXX XXX to XXXXXXX characters so that they match with the OS codepoint dataset

In [ ]:
mask = (SWW_postcode['Postcode'].str.len()>7)
mask2 = (SWW_postcode['Postcode'].str.len()<8)

In [ ]:
df1 = SWW_postcode.loc[mask]

In [ ]:
df2 = SWW_postcode.loc[mask2]

In [ ]:
df2

In [ ]:
df1['First']= df1['Postcode'].str[:4]

In [ ]:
df1['Last']= df1['Postcode'].str[5:8]

In [ ]:
df1['New_pstcd'] = df1['First'].map(str) + df1['Last']

In [ ]:
df1

In [ ]:
df1_n = df1[['date', 'New_pstcd', 'Nitrate']]

In [ ]:
df1_n = df1_n.rename(columns={'New_pstcd':'Postcode'})

In [ ]:
df_all = pd.concat([df1_n, df2])

In [ ]:
# Change the postcode's in the postcode column using the upper command so that all of the postcodes are capital
# and can match with the OS code point dataset
df_all['Postcode']=df_all['Postcode'].str.upper()

In [ ]:
df_all = df_all.set_index('Postcode')

In [ ]:
df_all

- read in the header file for the Code point postcode files 
- set the column headings to the first row (which contains the header names)
- set the index to the column which says postcode
- delete the row with the duplicate of column headings- so end up with a table of just the column headings

In [ ]:
labels2 = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

In [ ]:
labels2.columns = labels2.iloc[0]

In [ ]:
labels2

In [ ]:
labels2 = labels2.set_index('Postcode')

In [ ]:
labels2 = labels2.reindex(labels2.index.drop('Postcode'))

- merge the SWW postcodes and the labels to create a dataframe in which we can fill with the OS easting and northing information

In [ ]:
df_all = df_all.merge(labels2, how='left', left_index=True, right_index=True)

In [ ]:
df_all

- read in the labels data again so that the OS codepoint files can be merged with the column headings

In [ ]:
labels = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

- Create a list of the files within the codepoint folder (containing all the different postcodes) to use within the for loop

In [ ]:
files = glob.glob(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\*")

In [ ]:
files

- create a loop to insert data into the df_all dataset
    - read the postcode file in
    - merge the labels and postcode file so the postcode file has column headings 
    - set the column headings as actual column headings 
    - set the index to postcode
    - delete the duplicate postcode row
    - insert a column 'date' because the table we are filling has a date column
    - ensure that the date column is in datetime format
    - insert the postcode information into the SWW postcode dataset
    - delete all of the extra postcodes that aren't in the SWW postcode dataset
    - display that the postcode file has been processed 

In [ ]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.set_index(['Postcode'])
    data = data.reindex(data.index.drop('Postcode'))
    data.insert(0, 'date', 'NaN')
    data.insert(1,'Nitrate', 'NaN')
    data['date']= pd.to_datetime(data['date'])
    df_all = df_all.combine_first(data)
    df_all = df_all[pd.notnull(df_all['date'])]
    
    
    print('Processed {fname}'.format(fname=file))

- save the dataset

In [ ]:
df_all.to_csv(r'S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\df_postcode_nitrate.csv')

- rerunning the code on the dataset where O's and I's have been changed to 0's and 1's

In [ ]:
# Read in postcode dataset
post_data = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\df_postcode_nitrate.csv")

In [ ]:
post_data = post_data.set_index('Postcode')

In [ ]:
post_data

In [ ]:
files = glob.glob(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Data\CSV\*")

In [ ]:
files

In [ ]:
labels = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\codepo_gb\Doc\Code-Point_Open_Column_Headers.csv", header = None)

In [ ]:
for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.set_index(['Postcode'])
    data = data.reindex(data.index.drop('Postcode'))
    data.insert(0, 'date', 'NaN')
    data['date']= pd.to_datetime(data['date'])
    post_data = post_data.combine_first(data)
    post_data = post_data[pd.notnull(post_data['date'])]
    
    
    print('Processed {fname}'.format(fname=file))

In [ ]:
post_data

In [ ]:
post_data_c = post_data[[ 'Eastings', 'Nitrate', 'Northings', 'date']]

In [ ]:
post_data_c

In [ ]:
east_northing = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\SWW Nitrate 2011-PRESENT_northing_easting.csv")

In [ ]:
east_northing

In [ ]:
east_northing = east_northing.set_index(['Postcode'])

In [ ]:
all_SWW = pd.concat([post_data_c, east_northing])

In [ ]:
all_SWW

In [ ]:
all_SWW.to_csv(r'S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\all_SWW.csv')

- read in unmatched SWW data 
- match with the postcode district centroid files
- match with postcode csv

In [ ]:
unmatched = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\all_SWW_unmatched.csv")

In [ ]:
unmatched

In [ ]:
unmatched = unmatched.set_index(['Postcode_editted'])

In [ ]:
postcode_dist = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\Post_d_centroids.csv")

In [ ]:
postcode_dist

In [ ]:
postcode_dist = postcode_dist.rename(columns={'X_Coord':'Eastings', 'Y_Coord':'Northings','District':'Postcode_editted'})

In [ ]:
postcode_dist = postcode_dist.set_index(['Postcode_editted'])

In [ ]:
postcode_dist = postcode_dist[['Eastings', 'Northings']]

In [ ]:
postcode_dist.insert(0, 'Postcode', 'NaN')

In [ ]:
postcode_dist.insert(2, 'Nitrate', 'NaN')
postcode_dist.insert(4, 'date', 'NaN')

In [ ]:
postcode_dist['date']= pd.to_datetime(postcode_dist['date'])

In [ ]:
unmatched = unmatched.combine_first(postcode_dist)
unmatched = unmatched[pd.notnull(unmatched['date'])]

In [ ]:
unmatched

In [ ]:
postcode = pd.read_csv(r"S:\Projects\SAHSU_Share\Diabetes\Datasets\Postcodes\all_postcodes_cut.csv")

In [ ]:
postcode

In [ ]:
postcode = postcode.rename(columns={'Postcode':'Postcode_editted'})

In [ ]:
postcode = postcode.set_index(['Postcode_editted'])

In [ ]:
postcode.insert(0, 'Postcode', 'NaN')

In [ ]:
postcode.insert(2, 'Nitrate', 'NaN')
postcode.insert(4, 'date', 'NaN')

In [ ]:
postcode['date']= pd.to_datetime(postcode['date'])

In [ ]:
unmatched = unmatched.combine_first(postcode)
unmatched = unmatched[pd.notnull(unmatched['date'])]

In [ ]:
unmatched

In [ ]:
unmatched.to_csv(r'S:\Projects\SAHSU_Share\Diabetes\Datasets\Drinking_water\SouthWest\editted\all_SWW_unmatched_matched.csv')